<a href="https://colab.research.google.com/github/ShamsRupak/ai-doc-processing-suite/blob/main/Analyze_a_Scanned_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract pdf2image opencv-python
!apt install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (217 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
from google.colab import files

uploaded = files.upload()

Saving MTG_10009588.pdf to MTG_10009588.pdf


In [3]:
from pdf2image import convert_from_path
from PIL import Image
import os

# Convert uploaded PDF to image(s)
pdf_path = next(iter(uploaded))
images = convert_from_path(pdf_path)

# Save first page as image
image_path = "page.png"
images[0].save(image_path, 'PNG')

In [4]:
import cv2

# Load and preprocess image
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 11, 17, 17)  # Denoise
gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                             cv2.THRESH_BINARY, 11, 2)

cv2.imwrite("preprocessed.png", gray)

True

In [5]:
import pytesseract

custom_config = r'--oem 3 -l eng'
ocr_text = pytesseract.image_to_string(gray, config=custom_config)

print("OCR Extracted Text:\n", ocr_text)

OCR Extracted Text:
 - UO

MORTGAGE DOC.# 10009588

DOCUMENT NUMBER

RECORDED 06/28/2011 09:35AM
JOHN LA FAVE
NAME & RETURN ADDRESS REGISTER OF DEEDS
M&I Home Lending Solutions Milwaukee County, HII
Attn: Secondary Marketing AMOUNT: 30 00

4121 NW Urbandale Drive

Urbandale, IA 50322 FEE EXEMPT @:

PARCEL IDENTIFIER NUMBER
716-0027-6
[Space Above This Line For Recording Data]

FHA Case No
State of Wisconsin
581-4247085-703 :

MIN 100273100009309945

 

THIS MORTGAGE ("Security Instrument”) is given on June 20, 2011
The Mortgagor is KIMBERLY HOGAN, A Single Person,

("Borrower") This Security Instrument is given to Mortgage Electronic Registration Systems, Inc ("MERS"),
(solely as nominee for Lender, as heremafter defined, and Lender's successors and assigns), as mortgagee MERS 1s
orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026,
Flint, MI 48301-2026, tel (888) 679-MERS M&I Bank FSB .
("Lender") 1s organized and existing under the 

In [6]:
import re

# Remove newlines and fix common OCR issues
ocr_text = " ".join(ocr_text.split())
ocr_text = re.sub(r'\bLO@N\b', 'LOAN', ocr_text, flags=re.IGNORECASE)
ocr_text = re.sub(r'\bMORTG@GE\b', 'MORTGAGE', ocr_text, flags=re.IGNORECASE)
ocr_text = re.sub(r'[^\w\s$%.,/-]', '', ocr_text)  # Clean junk

print("Cleaned OCR Text:\n", ocr_text)

Cleaned OCR Text:
 - UO MORTGAGE DOC. 10009588 DOCUMENT NUMBER RECORDED 06/28/2011 0935AM JOHN LA FAVE NAME  RETURN ADDRESS REGISTER OF DEEDS MI Home Lending Solutions Milwaukee County, HII Attn Secondary Marketing AMOUNT 30 00 4121 NW Urbandale Drive Urbandale, IA 50322 FEE EXEMPT  PARCEL IDENTIFIER NUMBER 716-0027-6 Space Above This Line For Recording Data FHA Case No State of Wisconsin 581-4247085-703  MIN 100273100009309945 THIS MORTGAGE Security Instrument is given on June 20, 2011 The Mortgagor is KIMBERLY HOGAN, A Single Person, Borrower This Security Instrument is given to Mortgage Electronic Registration Systems, Inc MERS, solely as nominee for Lender, as heremafter defined, and Lenders successors and assigns, as mortgagee MERS 1s orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026, Flint, MI 48301-2026, tel 888 679-MERS MI Bank FSB . Lender 1s organized and existing under the laws of the United States of America , and has a

In [7]:
key_data = {}

patterns = {
    "loan_amount": r"Loan Amount[:\s]*\$?([\d,]+\.\d{2})",
    "interest_rate": r"Interest Rate[:\s]*([\d.]+%)",
    "lender": r"Lender[:\s]*([A-Za-z0-9 &,.]+)",
    "borrower": r"Borrower[:\s]*([A-Za-z0-9 &,.]+)",
    "property_address": r"Property Address[:\s]*(.+?)\s*(?=Date|Loan)",
    "recording_date": r"Recording Date[:\s]*([A-Za-z]+\s\d{1,2},\s\d{4})",
    "loan_date": r"Loan Date[:\s]*([A-Za-z]+\s\d{1,2},\s\d{4})",
    "document_title": r"(Mortgage|Deed of Trust)"
}

for field, pattern in patterns.items():
    match = re.search(pattern, ocr_text, re.IGNORECASE)
    if match:
        key_data[field] = match.group(1)

print("Extracted Fields:\n", key_data)

Extracted Fields:
 {'lender': ', as heremafter defined, and Lenders successors and assigns, as mortgagee MERS 1s orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026, Flint, MI 48301', 'borrower': 'This Security Instrument is given to Mortgage Electronic Registration Systems, Inc MERS, solely as nominee for Lender, as heremafter defined, and Lenders successors and assigns, as mortgagee MERS 1s orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026, Flint, MI 48301', 'document_title': 'MORTGAGE'}


In [8]:
ocr_data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

# Display sample bounding boxes
for i in range(5):
    print(f"Word: {ocr_data['text'][i]}, BBox: ({ocr_data['left'][i]}, {ocr_data['top'][i]}, {ocr_data['width'][i]}, {ocr_data['height'][i]})")

Word: , BBox: (0, 0, 1700, 2200)
Word: , BBox: (552, 40, 874, 73)
Word: , BBox: (552, 40, 874, 73)
Word: , BBox: (552, 40, 874, 73)
Word: -, BBox: (552, 82, 3, 5)


In [9]:
extracted_json = {}

for i in range(len(ocr_data['text'])):
    word = ocr_data['text'][i].strip().upper()
    for field, value in key_data.items():
        if value.upper() in word or word in value.upper():
            extracted_json[field] = {
                "text": value,
                "bbox": [ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i]],
                "conf": ocr_data['conf'][i]
            }

import json
print("Final Structured JSON Output:\n")
print(json.dumps(extracted_json, indent=2))

Final Structured JSON Output:

{
  "lender": {
    "text": ", as heremafter defined, and Lenders successors and assigns, as mortgagee MERS 1s orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026, Flint, MI 48301",
    "bbox": [
      754,
      2166,
      30,
      14
    ],
    "conf": 95
  },
  "borrower": {
    "text": "This Security Instrument is given to Mortgage Electronic Registration Systems, Inc MERS, solely as nominee for Lender, as heremafter defined, and Lenders successors and assigns, as mortgagee MERS 1s orgamzed and existing under the laws of Delaware, and has an address and telephone number of PO Box.2026, Flint, MI 48301",
    "bbox": [
      754,
      2166,
      30,
      14
    ],
    "conf": 95
  },
  "document_title": {
    "text": "MORTGAGE",
    "bbox": [
      438,
      2166,
      346,
      18
    ],
    "conf": -1
  }
}
